In [1]:
#required library
from music21 import *
import music21
import os
import glob
import re
import numpy as np
import json

In [2]:
#key transpose profile
MtoC={
    'Gb':-6,
    'F':-5,
    'E':-4,
    'D#':-3,
    'Eb':-3,
    'D':-2,
    'C#':-1,
    'Db':-1,
    'C':0,    
    'B':1,
    'A#':2,
    'Bb':2,
    'A':3,
    'G#':4,
    'Ab':4,
    'G':5,
    'F#':6
}
mtoA={
    'D#':6,
    'G#':5,
    'Ab':5,
    'C#':4,
    'F#':3,
    'B':2,
    'E':1,
    'A':0,
    'D':-1,
    'G':-2,
    'C':-3,
    'F':-4,
    'Bb':-5,
    'Eb':-6
}

In [3]:
#read file
c = converter.parse('./Chopin_F._Nocturne_in_E_Major,_Op.26_No.2.mxl')

In [4]:
#remove key signature
for e in c.recurse().getElementsByClass(key.KeySignature):
    while e.sharps!=0:
        e.transpose(-1,inPlace=True)

In [5]:
#remove key signature
for e in c.recurse().getElementsByClass(stream):
    print(e)

In [6]:
def cal_offset(e):
    if e is None:
        return 0
    return e.offset+cal_offset(e.activeSite)

In [7]:
#extract notes
all_notes=[]
for el in c.recurse().notes:
    if el.lyric is not None:
        el.lyric=el.lyric.replace('♭','b')
    
    #print(el)
    #print([el.getOffsetBySite(c),el.lyric, el, el.activeSite,el.offset, el.activeSite.offset])
    all_notes.append([el.lyric, el, cal_offset(el)])
    
    #DEBUG
    #print([el.lyric, el, cal_offset(el)])

In [8]:
#sort by first occurence
b = sorted(all_notes, key=lambda x: x[0] if x[0] is not None else "ZZZ")
b=sorted(all_notes,key=lambda x: x[-1])


#DEBUG
for e in b:
    print(e)

[None, <music21.note.Note B>, 0.0]
['EM(I)', <music21.note.Note G#>, 1.0]
[None, <music21.note.Note E>, 1.0]
[None, <music21.chord.Chord E3 G#3 C#4>, 2.0]
[None, <music21.note.Note A>, 2.5]
[None, <music21.note.Note B>, 3.0]
[None, <music21.chord.Chord B2 G#3 B3>, 3.0]
[None, <music21.note.Note D#>, 3.5]
[None, <music21.note.Note E>, 4.0]
[None, <music21.chord.Chord B3 E4 G#4>, 4.0]
[None, <music21.note.Note C#>, 4.5]
['V', <music21.note.Note B>, 5.0]
[None, <music21.note.Note B>, 5.0]
[None, <music21.chord.Chord F#3 D#4>, 6.0]
[None, <music21.note.Note F#>, 7.0]
[None, <music21.chord.Chord A3 D#4>, 7.0]
[None, <music21.chord.Chord B2 A3>, 8.0]
[None, <music21.note.Note G#>, 8.5]
['VI', <music21.note.Note E>, 9.0]
[None, <music21.chord.Chord C#2 C#3>, 9.0]
[None, <music21.chord.Chord E3 G#3 C#4>, 10.0]
['AM(V)', <music21.note.Note F#>, 11.0]
[None, <music21.chord.Chord B1 B2>, 11.0]
[None, <music21.note.Note G#>, 11.5]
[None, <music21.note.Note A>, 12.0]
[None, <music21.chord.Chord E3 

In [9]:
#main
current_key=None
Major=None
for e in b:
    if current_key is None:
        #fail-safe
        if e[1].lyric is None:
            continue
        print('start')
        e[1].addLyric('start')#every score should veriyf its start***
        current_key=e[1].lyric
        
        #identify major or minor
        if 'M' in current_key:
            Major=True        
        else:
            Major=False
                        
        #identify key name
        if Major:
            e[1].addLyric('trans->CM')
            current_key=current_key.split('M')[0]
        else:
            e[1].addLyric('trans->Am')
            current_key=current_key.split('m')[0]
         
    #check if key change
    elif e[1].lyric is not None and ('m' in e[1].lyric or 'M' in e[1].lyric) and '(' in e[1].lyric:  # '(' --> is just to make sure it is key change        
            
        #DEBUG
        #print(current_key,Major)
        #print(e[1].lyric)
        #print('----------------------')
        
        #check if key is different from previous key        
        if not((Major and 'M' in e[1].lyric and current_key==e[1].lyric.split('M')[0]) or (not Major and 'm' in e[1].lyric and current_key==e[1].lyric.split('m')[0])):
            #update key
            print('change key')
            current_key=e[1].lyric
            
            #identify major or minor
            if 'M' in current_key:
                Major=True
            else:
                Major=False
                
            #identify key name
            if Major:
                current_key=current_key.split('M')[0]
                e[1].addLyric('trans->CM')
            else:
                current_key=current_key.split('m')[0]
                e[1].addLyric('trans->Am')
            
            
    if Major:
        e[1].transpose(MtoC[current_key], inPlace=True)
    else:
        e[1].transpose(mtoA[current_key], inPlace=True)

start
change key
change key
change key
change key
change key
change key
change key
change key
change key
change key
change key
change key
change key
change key
change key
change key
change key
change key
change key
change key
change key
change key


In [10]:
#save
GEX = musicxml.m21ToXml.GeneralObjectExporter(c)
out = GEX.parse()
with open("test.musicxml", "wb") as f:
        f.write(out)

DurationException: Cannot return types smaller than 2048th; qLen was: 1/1680